In [1]:
import pickle
import pandas as pd
from src.data import get_hyperplane_datasets
from src.models import get_models
from utils import rank_logs, friedman_statistics, iman_davenport, run_prequential_eval

In [ ]:
INSTANCES = 1_000_000
N_DATASETS = 20
synth_datasets = get_hyperplane_datasets(N_DATASETS)
models = get_models()
N_MODELS = len(models)

In [ ]:
logs = run_prequential_eval(models, synth_datasets, INSTANCES)

In [ ]:
with open("./logs/hyperplane.pkl", "+wb") as file:
    pickle.dump(logs, file)

In [ ]:
params = ["performance", "memory", "time"]
results = []
for param in params:
    friedman_matrix = rank_logs(logs, param, models.keys(), synth_datasets.keys())
    results.append(friedman_matrix.rank(axis=1, ascending=True, method="average").mean())

In [ ]:
results = pd.DataFrame(results, index=params)
stats_results = results.copy()
stats_results["fried_stats"] = results.apply(lambda x: friedman_statistics(x, N_DATASETS), axis=1)
stats_results["iman_davenport"] = stats_results["fried_stats"].apply(lambda x: iman_davenport(x, N_DATASETS, N_MODELS))
stats_results